In [7]:
import sqlite3 as sql
import pandas as pd


conn = sql.connect("states.db")
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql(tables_query, conn)
table_names = tables_df['name'].tolist()[1:]


dataframes = {}
for table_name in table_names:
    dataframes[table_name] = pd.read_sql(f"SELECT * FROM {table_name}", conn)
conn.close()

In [8]:
unemployment_rate = dataframes['unemployment_rate']
state_survival_rates = dataframes['survival_rates']
industry_survival_rates = dataframes['industry_survival_rates']
establishments = dataframes['establishments']

In [9]:
unemployment_rate

,Year,Unemployment Rate,State
0,1994,6.175000,AL
1,1995,5.833333,AL
2,1996,5.208333,AL
3,1997,4.933333,AL
4,1998,4.458333,AL
...,...,...,...
1576,2020,5.858333,WY
1577,2021,4.541667,WY
1578,2022,3.416667,WY
1579,2023,2.916667,WY
